In [ ]:
!pip install selenium pandas

# WIP (needs to be adapted to each rugfarm)

In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from io import StringIO
import tempfile

In [ ]:
# Get a selenium driver with Cloudflare bypass.
# tmpdirname: Temporary directory for downloads.
# url: URL to scrape. The url content doesn't really matter. We just need to use one that is in the same domain as the target URL.
def get_driver(tmpdirname, url):
    options = Options()
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("prefs", {
        "download.default_directory": tmpdirname,
        "download.prompt_for_download": False,
    })

    service = Service('/usr/bin/chromedriver')  # Adjust path to your chromedriver
    driver = webdriver.Chrome(service=service, options=options)
    

    # Cloudflare bypass
    # IMPORTANT. Solve the Cloudflare challenge manually, then open a new tab with the same URL and solve the challenge again.
    # After that, the script will run without any issues. (you can close the second tab if you want)
    driver.get(url)

    # Wait time for the user to solve the challenge manually. Change it if needed.
    time.sleep(1)

    return driver

In [ ]:
def get_transfers(driver: webdriver.Chrome, tmpdirname, wallet_address, activity_type=None):
    url = f"https://api-v2.solscan.io/v2/account/transfer/export?address={wallet_address}" + (f"&activity_type[]={activity_type}" if activity_type else "")

    # Get files that are already in the directory
    files = os.listdir(tmpdirname)

    # clear the window to avoid getting last transfer data
    driver.get("about:blank")
    driver.get(url)
    # Sometimes solscan doesn't respond with a download, but instead a CSV in the body, so we need to check for that
    
    raw_csv = driver.find_element("tag name", "body").get_attribute("innerHTML")
        
    if(raw_csv == ""):
        # Get the latest downloaded file comparing with the files before the download and wait for it to be downloaded
        new_files = os.listdir(tmpdirname)
        downloaded_file = list(set(new_files) - set(files))[0]
        
        # Wait for the file to be downloaded
        while downloaded_file is None:
            new_files = os.listdir(tmpdirname)
            downloaded_file = list(set(new_files) - set(files))[0]
            # time.sleep(0.1)

        # Check if file is still being downloaded or corrupted
        while True:
            try:
                # Read the downloaded file
                df = pd.read_csv(f"{tmpdirname}/{downloaded_file}")
                break
            except Exception as e:
                print(e)
                time.sleep(0.1)
                continue
    else:
        df = pd.read_csv(StringIO(raw_csv))

    return df

## St3

In [ ]:
# Check if wallet is a transfer wallet and returns the previous creator wallet if it is
def is_bridge_wallet(wallet_transfers: pd.DataFrame):
    if(len(wallet_transfers) > 6):
        return False
    elif(len(wallet_transfers) > 0 and len(wallet_transfers) <= 6):
        return True
    else:
        raise Exception("No transfers found")

def get_created_token(wallet_transfers):
    wallet_transfers = wallet_transfers[(wallet_transfers["Action"] == "MINT") & (wallet_transfers["Amount"] == 10000000000000000)]
    if(len(wallet_transfers) == 0):
        return None
    return wallet_transfers.iloc[0]["TokenAddress"]

def get_prev_wallet(wallet_transfers):
    return wallet_transfers.iloc[-1]["From"]

# Name of the farm
farm_name = "st3"
# first wallet to analyze
start_wallet = "9AhFgzwQteTNE4h34x5ZKhHT2Ctpai2kmEBNq8PEXDZp"
# limit of wallets to analyze
limit = 100

# Create tokens.txt file if it doesn't exist
if not os.path.exists(f"{farm_name}/tokens.txt"):
    with open(f"{farm_name}/tokens.txt", "w") as f:
        pass

with tempfile.TemporaryDirectory() as tmpdirname:
    driver = get_driver(tmpdirname,f"https://api-v2.solscan.io/v2/account/transfer/export?address={start_wallet}")

    wallet = start_wallet
    while wallet != "CRBYGyfcRSiwcpUr4qxbVeR7MDNb32mkhxxzFAN7iinS":
        wallet_transfers = get_transfers(driver,tmpdirname,wallet)
        if(is_bridge_wallet(wallet_transfers)):
            print(f"Bridge wallet {wallet}")
        else:
            print(f"Creator wallet {wallet}")
            token = get_created_token(wallet_transfers)
            if token:
                print(f"Token: {token}")
                print("\n")
                with open(f"{farm_name}/tokens.txt", "a") as f:
                    f.write(f"{token}\n")
                
            else:
                print("No token found in creator wallet. Stoping collection.")
                break
                
            

        wallet = get_prev_wallet(wallet_transfers)
    
    driver.quit()